# STM32 Model Analysis with STM32TFLM

Analyze trained TFLite models using STM32TFLM for accurate memory usage.

In [3]:
import os
from pathlib import Path
from stm32_model_analyzer import analyze_trained_models, compare_with_constraints

# Check STM32TFLM
stm32tflm_path = "./stm32tflm"
if os.path.exists(stm32tflm_path):
    print("✅ STM32TFLM found")
else:
    print("❌ STM32TFLM not found")
    print("Download from: https://www.st.com/en/embedded-software/x-cube-ai.html")

✅ STM32TFLM found


In [5]:
# Configuration
MODELS_DIR = "./Workspace/trained_models"
STM32TFLM_PATH = "./Workspace/stm32tflm"

# STM32L412KBU3 constraints
MAX_RAM = 40960    # 40 KB
MAX_FLASH = 131072 # 128 KB

print(f"Models: {MODELS_DIR}")
print(f"Limits: {MAX_RAM/1024:.0f} KB RAM, {MAX_FLASH/1024:.0f} KB Flash")

Models: ./Workspace/trained_models
Limits: 40 KB RAM, 128 KB Flash


In [6]:
# Analyze models
print("🔍 Starting analysis...")
results = analyze_trained_models(MODELS_DIR, STM32TFLM_PATH)

if not results:
    print("No models found or analysis failed")

🔍 Starting analysis...


FileNotFoundError: STM32TFLM not found at ./Workspace/stm32tflm

In [ ]:
# Check constraints
if results:
    feasible = compare_with_constraints(results, MAX_RAM, MAX_FLASH)
    
    if feasible:
        best = min(feasible, key=lambda x: x['ram_bytes'] + x['flash_bytes'])
        print(f"\n🏆 Best: {best['model_name']}")
        print(f"Flash: {best['flash_bytes']/1024:.2f} KB")
        print(f"RAM: {best['ram_bytes']/1024:.2f} KB")
    else:
        print("\n⚠️ No feasible models")

In [ ]:
# Export results
if results:
    import pandas as pd
    
    df = pd.DataFrame(results)
    df['flash_kb'] = df['flash_bytes'] / 1024
    df['ram_kb'] = df['ram_bytes'] / 1024
    df['feasible'] = (df['flash_bytes'] <= MAX_FLASH) & (df['ram_bytes'] <= MAX_RAM)
    
    df.to_csv('stm32_results.csv', index=False)
    print("📊 Results saved to stm32_results.csv")
    print(df[['model_name', 'flash_kb', 'ram_kb', 'feasible']])